### Librerias

In [3]:
#instalacion
!pip install --upgrade langchain langchain_openai gradio chromadb pypdf ipywidgets langchain_community

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate 
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import gradio as gr

### Fuentes RAG

In [25]:
url = 'https://www.madrid.es/UnidadesDescentralizadas/Educacion_Ambiental/ContenidosBasicos/Publicaciones/Residuos_Limpieza/GuiaReducirResiduos/GuíaBuenasPrácticasReducirResiduos.pdf'
url2 = 'DATA/CS-RecogidaResiduos_Folleto2024.pdf'
url3 = 'https://www.madrid.es/UnidadesDescentralizadas/UDCEstadistica/Nuevaweb/Publicaciones/anuesta/nuevos/Anuario%20Estadístico%20Municipal/Anuario%20estadístico%202023/Capítulos/CAPITULO%206.pdf'
url4 = 'https://www.icv.csic.es/prevencion/Documentos/residuos/puntos.pdf'
url5 =  'https://www.redcimas.org/wordpress/wp-content/uploads/2013/02/gestionresiduos_comparativa.pdf'
url6 = 'https://conerrede.es/wp-content/uploads/2022/08/dossier-informativo_ayto-madrid.pdf'


### Configuracion

In [26]:
API_KEY = ''
llm = ChatOpenAI(model='gpt-3.5-turbo-0125' , temperature= 0 , api_key=API_KEY)
emmbeding = OpenAIEmbeddings(api_key= API_KEY)

### Corpus

In [27]:
urls = [url, url2, url3, url4,url5,url6] #un libro de libros

In [29]:

pages = []
for url in urls:
    pdf = PyPDFLoader(url)
    pages += pdf.load_and_split()

In [30]:
len(pages)

224

In [31]:
pages[290]

IndexError: list index out of range

# Vectorstore 

In [32]:

vectorstore = Chroma.from_documents(
    documents=pages,
    embedding=emmbeding,
)

In [33]:

retriever = vectorstore.as_retriever()

# Prompt Template

In [35]:
 template = """
          ###INSTRUCCIONES:
          Eres un asistente virtual en tareas de concienciacion del reciclaje y las buenas practicas medioambientales y tu objetivo es recopilar y dar respuesta segun la informacion que tienes a las preguntas que te planteen los usuarios.Seimpre con un tono amigable y pedagogico.

          En tu respuesta, SIEMPRE:
          (0)Se un lector atento a los detalles: Lee la pregunta y el contexto, entiende ambos antes de responder
          (1)Tu respuesta comienza con un tono amigable, asegúrate de que entiendes la pregunta del usuario correctamente, en caso negativo, pregunta al usuario para obtener más información.
          (2)Si el contexto te permite responder a la pregunta, escribe una respuesta detallada, útil y fácil de entender, CON FUENTES REFERENCIADAS EN EL TEXTO. SI NO: no puedes encontrar la respuesta, responde con una explicación, comenzando con: "No pude encontrar la información en los documentos a los que tengo acceso".
          (3)Debajo de tu respuesta, proporciona un enlace a la fuente referenciada( Es decir, párrafos legales, artículos, etc.) y una breve explicación de por qué la fuente es relevante.
          (4)Ahora que ya tienes tu respuesta, revísala para asegurarte de que sea clara, precisa y educada y cumple con los puntos anteriores.

          ###PIENSA PASO POR PASO

            Responde a la siguiente pregunta utilizando el contexto proporcionado.
              ### Question: {question} ###
              ### Context: {context} ###
              ### Respuesta 'util con Fuentes:        
"""

prompt = PromptTemplate.from_template(template)

## Chain

In [36]:
chain = (
    {'context':retriever,
     'question':RunnablePassthrough()}
     | prompt
     | llm
     | StrOutputParser()
)
 

In [37]:
def get_asnwer(question):
    return chain.invoke(question)

In [38]:
get_asnwer('donde puedo tirar resoduos organicos')

'¡Hola! Para desechar residuos orgánicos, como restos de alimentos, posos de café, hojas y ramas de plantas, entre otros, debes depositarlos en el contenedor gris con tapa naranja. Este contenedor está destinado para los restos orgánicos y otros residuos domésticos que no se incluyen en contenedores específicos.\n\nLa materia orgánica depositada en este contenedor se recoge para su posterior tratamiento, ya sea en centros de compostaje a nivel doméstico o en centros de tratamiento especializados, como el Parque Tecnológico de Valdemingómez, donde se transforma en compost de alta calidad, un abono orgánico beneficioso para el suelo.\n\nRecuerda que es importante separar adecuadamente los residuos para facilitar su reciclaje y tratamiento adecuado. ¡Cuidar el medio ambiente es responsabilidad de todos!\n\nFuente: Guía de Buenas Prácticas para la Reducción de Residuos del Ayuntamiento de Madrid.\n\nEnlace: https://www.madrid.es/UnidadesDescentralizadas/Educacion_Ambiental/ContenidosBasico

In [39]:
iface = gr.Interface(fn=get_asnwer, inputs=gr.Textbox(
    value="Pregúntame lo que necesites"),
    live=False, 
    outputs="markdown",  
    title="Inteligencia sobre documentación mediante LLM",
    description="Puedes preguntar cualquier pregunta sobre migración",
    theme=gr.themes.Soft(),
    allow_flagging="never",)

iface.launch()

/Users/iborja/.pyenv/versions/3.12.6/lib/python3.12/site-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
